# The Oregonator Model

[AMath 586, Spring Quarter 2016](http://faculty.washington.edu/rjl/classes/am586s2016/)

This is a simple ODE model for the [Belousov Zhabotinsky reaction](http://www.scholarpedia.org/article/Belousov-Zhabotinsky_reaction).


In [ ]:
%pylab inline

The chemical reactions are most dramatic when they vary in space, as seen for example in this video:

In [ ]:
from IPython.display import YouTubeVideo

YouTubeVideo('3JAqrRnKFHo')

Modelling these time-dependent spatial patterns can be done by solving reaction-diffusion equations, partial differential equations in space and time that we will study later in the quarter.

If the reactions take place in a well-stirred container, then oscillatory changes in color can be observed that are spatially uniform and vary only in time, and hence can be modelled with time-dependent ODEs.  We will study ODEs coming from chemical kinetics as a good model problem for numerical methods (see Section 7.4 of the book).

In [ ]:
YouTubeVideo('UVRGD5xokY8')

## Oregonator model

This model was developed by Field and Noyes at the University of Oregon in 1974, see for example http://www.scholarpedia.org/article/Oregonator

$$
\begin{split}
\epsilon_1 u_0\prime(t) &= qu_1(t) - u_0(t)u_1(t) + u_0(t)(1 - u_0(t)) \\
\epsilon_2 u_1\prime(t) &= -qu_1(t) - u_0(t)u_1(t) + \phi u_2(t)\\
u_2\prime(t) &= u_0(t) - u_2(t)
\end{split}
$$

Note that the vector $u(t)$ has been written with components $u(t) = [u_0(t),~ u_1(t),~ u_2(t)]$  to match the Python convention of 0-based indexing.

### Numerical solution:

We set up the function to compute $f(u,t)$ for this system, with one particular choice of the parameters:

In [ ]:
e1 = 0.1 
e2 = 0.1 
q = 0.01 
phi = 1.

def f(u,t):
    f0 = 1./e1 * (q*u[1] - u[0]*u[1] + u[0]*(1.-u[0]))
    f1 = 1./e2 * (-q*u[1] - u[0]*u[1] + phi*u[2])
    f2 = u[0] - u[2]
    f = array([f0, f1, f2])  # return as array rather than list to use in Euler below.
    return f

Set `tout` to be the vector of desired output times, and `eta` to be the initial conditions at time 0:

In [ ]:
tout = linspace(0,50,501)
eta = array([1., 1., 1.])

### Use an ODE solver from SciPy:

We first import the solver and then use it.

For documentation, see http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html

In [ ]:
from scipy.integrate import odeint

In [ ]:
u = odeint(f, eta, tout)

Now plot the solution, using a logarithmic scale so that the oscillations can be seen more clearly.  Label each line so that we can tell which component is which:

In [ ]:
figure(figsize=(12,5))  # make the figure wider than the default
semilogy(tout,u[:,0], label='u0')
semilogy(tout,u[:,1], label='u1')
semilogy(tout,u[:,2], label='u2')
xlim(0,60)  # make room for the legend
legend()  # uses the labels from the plot commands

You can get information about how the ODE solver worked on this problem by setting the `full_output` parameter to `True`, in which case it returns the solution and also a Python dictionary of information.  

In [ ]:
u, infodict = odeint(f, eta, tout, full_output=True)

For example, `infodict['hu']` contains the time step used in each step and `infodict['nfe']` the cumulative number of `f` evaluations.  See http://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.odeint.html
for more details.

In [ ]:
figure(figsize=(12,5))
subplot(1,2,1)
plot(infodict['hu'])
title('size of time step in each step')
subplot(1,2,2)
plot(infodict['nfe'])
title('Cumulative number of f evals')

## Forward Euler method

Here's a simple implementation of Forward Euler for this same problem:

In [ ]:
t0 = tout[0]
tfinal = tout[-1]

def euler(nsteps):
    t = linspace(t0, tfinal, nsteps+1)
    dt = t[1] - t[0]
    U = empty((nsteps+1,3))  # array for computed solution
    U[0,:] = eta
    for n in range(nsteps):
        U[n+1,:] = U[n,:] + dt * f(U[n,:], t[n])
        
    figure(figsize=(12,5))
    semilogy(t, U[:,0], label='u0')
    semilogy(t, U[:,1], label='u1')
    semilogy(t, U[:,2], label='u2')
    xlim(0,60)  # make room for the legend
    legend()  # uses the labels from the plot commands
    title('%i steps, dt = %7.4f' % (nsteps, dt))

With 1500 steps the solution looks pretty good, but if the number of steps is reduced to say 1200, (large $\Delta t$) then it blows up.

In [ ]:
euler(1500)
plot(tout, u[:,1],'k',label='odeint u1')
legend()